# Modular RAG  - Raptor RAG (2024, Stanford Univ.)

- Tackling Problem: RAG에 들어가는 문서 수가 많아질수록, 그리고 답안에 필요한 정보량이 long-tail로 방대해야 할때 Retrieval 성능이 너무 좋지 않아진다

- 제안 방법론: 정보들을 단계적 추상화를 통해 enrich시켜서 질문에서 요구하는 context의 뎁스에 알맞는 청크가 뽑히도록 하자.

- (인덱싱 프로세스) Text Chunk를 의미적으로 유사한 다른 Chunk들과 군집화 시키고 -> 군집별 요약문 생성하여 정보 압축 -> 요약문에 대한 임베딩 생성 (전체 프로세스 n번 반복하여 계층적 트리구조 생성)

- (검색 프로세스) Query와 가장 유사한 정보를 Root Node(Layer)에서 찾고, 그 root node가 생성한 요약문이 기반으로 두는 child summary를 찾고, 반복하며 최종적으로 사용자가 지정한 n개의 top-k 유사한 노드가 뽑힐때까지 트리 구조를 traverse함.

# RAPTOR RAG 구조

### Indexing 구조

- 특이점은, GMM기반의 군집화를 하기 때문에 클러스터별 해당하는 노드의 개수가 다르고, 각 차일드노드가 속할 수 있는 패런트노드도 1대다수 관계일 수 있는데, 정보체계라는걸 생각해보면 이 클러스터링 알고리즘이 굉장히 적합한 것으로 사료됨.

# Modular RAG(RAPTOR) 간단구성

- 관련 패키지 임포트
- 활용 LLM api정보 설정
- RAG 파이프라인 글로벌 세팅 설정
- 기준 데이터셋 로드
- 기준 데이터셋 전처리
- (베이스RAG)벡터스토어인덱스 설정
- (베이스RAG)쿼리 엔진 설정
- (RAPTOR RAG) 크로마DB 설정
- (RAPTOR RAG) 쿼리 엔진 설정

In [2]:
# 관련 패키지 임포트
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex
)
import os
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.settings import Settings
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
from llama_index.packs.raptor import RaptorPack
from llama_index.core.query_engine import RetrieverQueryEngine
import nest_asyncio
nest_asyncio.apply()

In [3]:
# RAG 파이프라인 글로벌 설정
Settings.llm=OpenAI(model='gpt-4o-mini',temperature=0)
Settings.embed_model=OpenAIEmbedding(model='text-embedding-3-small')

In [4]:
# 데이터 로드
dataset = SimpleDirectoryReader(input_files=['./cinderella.txt']).load_data()

In [5]:
# 청크사이즈 전처리 파이프라인 생성
pipeline_200 = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=300, chunk_overlap=0)
    ]
)

# 설정한 파이프라인 설정으로 기준 다큐먼트 노드 오브젝트로 변경
nodes_200 = pipeline_200.run(documents=dataset)

In [6]:
#기준 데이터 확인
nodes_200

[TextNode(id_='d9504d46-e0cd-4bec-b1e2-5edffe3e83dd', embedding=None, metadata={'file_path': 'cinderella.txt', 'file_name': 'cinderella.txt', 'file_type': 'text/plain', 'file_size': 20547, 'creation_date': '2025-05-01', 'last_modified_date': '2025-05-01'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='c370b4a9-168a-4c6d-bb35-101ac1cd9a0c', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': 'cinderella.txt', 'file_name': 'cinderella.txt', 'file_type': 'text/plain', 'file_size': 20547, 'creation_date': '2025-05-01', 'last_modified_date': '2025-05-01'}, hash='f7ccb35b5e68f7e345e47d667a2ca3f87603fe7a8959cbd416d7a832f1a8f277'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='6f2ef8da-6d33-4d33

# Naive-RAG 로 베이스 모델 구축

In [7]:
# 벡터스토어 인덱스 설정
vector_index_200 = VectorStoreIndex(nodes_200)

# 쿼리 엔진 설정
query_engine_200 = vector_index_200.as_query_engine(similarity_top_k=2)

In [8]:
from pprint import pprint

In [9]:
#질문: "How does Cinderella find a happy ending?"
response = query_engine_200.query("How does Cinderella find a happy ending?")


In [10]:
pprint(response.response)

('Cinderella finds a happy ending through her kindness and virtue, which '
 'ultimately lead to her being recognized by the prince. Despite her initial '
 'hardships and the challenges posed by her stepsisters, her good heart and '
 'the magical assistance of her fairy godmother allow her to attend the ball, '
 "where she captures the prince's attention. The story suggests that her true "
 'worth is acknowledged, and the little glass slipper serves as a symbol of '
 'her identity and connection to the prince, paving the way for their reunion '
 'and a joyful conclusion to her struggles.')


In [11]:
# hallucination이 없을까?
# 참조 컨텍스트 확인
pprint(response.source_nodes[0].text)
pprint(response.source_nodes[1].text)

('Cinderella reached home quite out of breath, without either coach or\n'
 'footmen, and with only her shabby clothes on her back; nothing, in\n'
 'short, remained of her recent magnificence, save a little glass slipper,\n'
 'the fellow to the one she had lost.\n'
 '\n'
 'The sentinels at the palace gate were closely questioned as to whether\n'
 'they had not seen a princess coming out; but they answered they had seen\n'
 'no one except a shabbily dressed girl, who appeared to be a peasant\n'
 'rather than a young lady.\n'
 '\n'
 'On this second night, as you have taken notice, dazzled by worldly show\n'
 'and the pleasing flattery of her royal lover, Cinderella over-stays her\n'
 "time, and is compelled to make her way back to her father's house on\n"
 'foot and in rags--an everlasting lesson to all the pretty little\n'
 'Cinderellas in the world to keep their word, and to act in good faith by\n'
 'such as befriend them. Never mind--her heart is in the right place--she\n'
 'is a charm

In [12]:
# 일단 retrieved context로만 답하게 만들어보자
response = query_engine_200.query(""" You have a long term memory issue. You are not able to answer anything unless the context is provided. When user asks the question, you will be provided with 2 context chunks that are relevant to answer the questions.
                                  answer the question based on the context provided strictly.

                                  Question:
                                  How does Cinderella find a happy ending?
                                  Answer:
                                  """)

In [13]:
pprint(response.response)

("Cinderella's journey leads her to a happy ending through the recognition of "
 'her true worth and the eventual discovery of the glass slipper she left '
 'behind. Despite her hardships and the rags she wears, her virtue and '
 'kindness shine through, ultimately allowing her to be reunited with her '
 'royal lover, who is searching for the mysterious princess. The glass slipper '
 'serves as a token of her identity, ensuring that she is traced back to her '
 'rightful place.')


In [14]:
# 참조 컨텍스트 확인
pprint(response.source_nodes[0].text)
pprint(response.source_nodes[1].text)

('Cinderella reached home quite out of breath, without either coach or\n'
 'footmen, and with only her shabby clothes on her back; nothing, in\n'
 'short, remained of her recent magnificence, save a little glass slipper,\n'
 'the fellow to the one she had lost.\n'
 '\n'
 'The sentinels at the palace gate were closely questioned as to whether\n'
 'they had not seen a princess coming out; but they answered they had seen\n'
 'no one except a shabbily dressed girl, who appeared to be a peasant\n'
 'rather than a young lady.\n'
 '\n'
 'On this second night, as you have taken notice, dazzled by worldly show\n'
 'and the pleasing flattery of her royal lover, Cinderella over-stays her\n'
 "time, and is compelled to make her way back to her father's house on\n"
 'foot and in rags--an everlasting lesson to all the pretty little\n'
 'Cinderellas in the world to keep their word, and to act in good faith by\n'
 'such as befriend them. Never mind--her heart is in the right place--she\n'
 'is a charm

# RAPTOR RAG로 QA 성능 고도화

In [15]:
# RAPTOR RAG 구현용 크로마DB 활용
client = chromadb.PersistentClient(path="./cinderella_db")
collection = client.get_or_create_collection('cinderella')

vector_store = ChromaVectorStore(chroma_collection=collection)

In [16]:
# 라마인덱스 랩터팩(통합팩) 사용하여 아키 세팅
raptor_pack = RaptorPack(
    dataset, #llamaindex document
    embed_model=OpenAIEmbedding(model='text-embedding-3-small'), #cluster별 summary text 임베딩 시 활용 모델
    llm=OpenAI(model='gpt-4o-mini', temperature=0), #Cluster별 Summary Text 생성할 LLM
    vector_store=vector_store, #기반 벡터스토어
    similarity_top_k=2, #추후 Retrieve 시 옵션
    mode="collapsed", #Tree구조, Collapse구조 선택
    transformations=[
        SentenceSplitter(chunk_size=300, chunk_overlap=0)
    ],
)

Generating embeddings for level 0.
Performing clustering for level 0.


/Users/hyeonjinho/.pyenv/versions/3.11.6/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Generating summaries for level 0 with 4 clusters.
Level 0 created summaries/clusters: 4
Generating embeddings for level 1.
Performing clustering for level 1.
Generating summaries for level 1 with 1 clusters.
Level 1 created summaries/clusters: 1
Generating embeddings for level 2.
Performing clustering for level 2.
Generating summaries for level 2 with 1 clusters.
Level 2 created summaries/clusters: 1


In [17]:
# 생성된 인덱스 기반 Raptor Retriever 엔진 정의
raptor_retriever = raptor_pack.retriever

raptor_query_engine = RetrieverQueryEngine.from_args(
    raptor_retriever, llm=OpenAI(model="gpt-4o-mini", temperature=0)
)

In [18]:
#질문: "How does Cinderella find a happy ending?"
response =raptor_query_engine.query("How does Cinderella find a happy ending?")

In [19]:
pprint(response.response)

('Cinderella finds a happy ending by marrying the prince after enchanting him '
 'at the royal ball. Despite the mistreatment from her stepmother and '
 'stepsisters, she remains kind-hearted and forgiving. After the prince vows '
 'to find the owner of the glass slipper she dropped, only Cinderella can wear '
 'it perfectly. She reveals the matching slipper she had hidden, which leads '
 'to her transformation into an even more magnificent version of herself. In a '
 'display of her gracious nature, she forgives her stepsisters and allows them '
 'to live in the palace, marrying them off to two lords. This culmination of '
 'events highlights her virtue and kindness, leading to her joyful union with '
 'the prince.')


In [20]:
# 참조 컨텍스트 확인
pprint(response.source_nodes[0].text)
pprint(response.source_nodes[1].text)

('Cinderella is a kind young woman who endures mistreatment from her cruel '
 "stepmother and stepsisters after her mother's death. Forced to do all the "
 'household chores while her stepsisters enjoy luxury, she remains patient and '
 "good-natured. When the king's son hosts a royal ball, Cinderella is mocked "
 'and excluded but helps her stepsisters prepare. She confides in her fairy '
 'godmother, who magically transforms a pumpkin into a carriage and gives her '
 'a beautiful gown and glass slippers, with a warning to return by midnight.\n'
 '\n'
 'At the ball, Cinderella enchants the prince but leaves at midnight, '
 "accidentally dropping a glass slipper. The prince vows to find the slipper's "
 "owner. Despite her stepsisters' attempts, only Cinderella can wear it "
 'perfectly. She reveals the matching slipper she had hidden, transforming '
 'into an even more magnificent version of herself. Her stepsisters ask for '
 'forgiveness, which she graciously grants. Ultimately, Cin

# DO IT YOURSELF
- korean_webtext.csv 데이터 사용
- RaptorRAG Pipeline 구성

In [22]:
import pandas as pd
data = pd.read_csv('./korean_webtext.csv').iloc[:,1:]

# Document 오브젝트로 변환
from llama_index.core import Document, VectorStoreIndex
documents = []

#Iterative하게 Document 만들기
for i, row in data.iterrows():
    documents.append(Document(
        text=row['text'],
        # extra_info={'title': row['title']}
    ))

In [23]:
# RAPTOR RAG 구현용 크로마DB
client = chromadb.PersistentClient(path="./wiki_db")
collection = client.get_or_create_collection("wiki")

vector_store = ChromaVectorStore(chroma_collection=collection)

In [ ]:
# 라마인덱스 랩터팩(통합팩) 사용하여 아키 세팅
raptor_pack = RaptorPack(
    documents, #llamaindex document
    embed_model=OpenAIEmbedding(model="text-embedding-3-small"), #cluster별 summary text 임베딩 시 활용 모델
    llm= OpenAI(model="gpt-4o-mini",temperature=0), #Cluster별 Summary Text 생성할 LLM
    vector_store=vector_store, #기반 벡터스토어
    similarity_top_k=2, #추후 Retrieve 시 옵션
    mode="collapsed", #Tree구조, Collapse구조 선택
    transformations=[
        SentenceSplitter(chunk_size=200, chunk_overlap=0)
    ],
)

In [25]:
# 생성된 인덱스 기반 Raptor Retriever 및 쿼리엔진 연결
raptor_retriever = raptor_pack.retriever

raptor_query_engine = RetrieverQueryEngine.from_args(
    raptor_retriever, llm=OpenAI(model='gpt-4o-mini', temperature=0)
)

In [26]:
#Naive RAG용
# 청크사이즈 전처리 파이프라인 생성
pipeline_200 = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=200, chunk_overlap=0)
    ]
)

# 설정한 파이프라인 설정으로 기준 다큐먼트 노드 오브젝트로 변경
nodes_200 = pipeline_200.run(documents=documents)

In [27]:
# 벡터스토어 인덱스 설정
vector_index_200 = VectorStoreIndex(nodes_200)

# 쿼리 엔진 설정
query_engine_200 = vector_index_200.as_query_engine(similarity_top_k=2)

In [28]:
# Naive RAG 답안 뽑기:"분당과 같은 신도시가 해외에서도 성공할 수 없는 이유가 뭘까?"
naive_response = query_engine_200.query("분당과 같은 신도시가 해외에서도 성공할 수 없는 이유가 뭘까?")

In [29]:
pprint(naive_response.response)

('해외에서 분당과 같은 신도시가 성공하기 어려운 이유는 현지의 문화, 시장, 사람들, 관계, 제도 등을 충분히 이해하지 못하기 때문입니다. '
 '국내식 접근 방식으로는 로컬 환경에 적합하지 않으며, 각 지역의 특성을 반영한 맞춤형 전략이 필요합니다. 성공적인 신도시 개발은 이러한 '
 '로컬 이해를 바탕으로 이루어져야 합니다.')


In [30]:
pprint(naive_response.source_nodes[0].text)
pprint(naive_response.source_nodes[1].text)

'그런데 어디에도 제대로된 한국적 신도시 성공담이 들리지 않는다. 오히려 고전을 면치못하는 게 현실이다.'
('우리나라가 아무리 국내에서 많은 신도시를 지었다고 해도 해외신도시개발은 다른 얘기다. 결코 만만하게 봐서는 안될것이다. 로마에 가면 '
 '로마법을 따라야하듯 해외에서 국내식 접근은 잊기 바란다. 로컬을 잘 이해하는 것이 성공의 첩경이다. 진출할 도시의 사람, 문화, 시장, '
 '관계, 제도 등을 잘 이해하는 것이 큰 사업의 가장 기초이자 성공의 결정요인이다.')


In [31]:
# RAPTOR RAG 답안 뽑기:"분당과 같은 신도시가 해외에서도 성공할 수 없는 이유가 뭘까?"
response =raptor_query_engine.query("분당과 같은 신도시가 해외에서도 성공할 수 없는 이유가 뭘까?")

In [32]:
pprint(response.response)

('해외에서 분당과 같은 신도시가 성공하기 어려운 이유는 현지의 사람, 문화, 시장, 관계, 제도 등을 충분히 이해하지 못하기 때문입니다. '
 '국내식 접근 방식으로는 로컬 환경에 적합한 해결책을 찾기 힘들고, 각 지역의 특성을 반영하지 않으면 성공적인 개발이 어렵습니다.')


In [33]:
# 참조 컨텍스트 확인
pprint(response.source_nodes[0].text)
pprint(response.source_nodes[1].text)

'그런데 어디에도 제대로된 한국적 신도시 성공담이 들리지 않는다. 오히려 고전을 면치못하는 게 현실이다.'
('우리나라가 아무리 국내에서 많은 신도시를 지었다고 해도 해외신도시개발은 다른 얘기다. 결코 만만하게 봐서는 안될것이다. 로마에 가면 '
 '로마법을 따라야하듯 해외에서 국내식 접근은 잊기 바란다. 로컬을 잘 이해하는 것이 성공의 첩경이다. 진출할 도시의 사람, 문화, 시장, '
 '관계, 제도 등을 잘 이해하는 것이 큰 사업의 가장 기초이자 성공의 결정요인이다.')


In [34]:
# Naive RAG 답안 뽑기:"캐나다가 주목받는 국가인 이유에 대해 알려줘"
naive_response = query_engine_200.query("캐나다가 주목받는 국가인 이유에 대해 알려줘")

In [35]:
pprint(naive_response.response)

('캐나다는 살기 좋은 나라와 유학하기 가장 좋은 나라로 높은 평가를 받고 있으며, 일, 방문, 생활 측면에서도 최고의 국가로 자리매김하고 '
 '있습니다. 이러한 이유로 많은 사람들이 캐나다를 주목하고 있습니다. 또한, 캐나다의 우수한 교육 프로그램, 아름다운 자연, 다문화 사회, '
 '저렴한 생활비 등이 유학생들에게 매력적으로 작용하고 있습니다.')


In [36]:
pprint(naive_response.source_nodes[0].text)
pprint(naive_response.source_nodes[1].text)

('캐나다는 3 년 연속 지구상에서 가장 주목할만한 국가로 선포되었습니다. 일반 타이틀을 집으로 가져 왔을뿐만 아니라 캐나다는 "살기 좋은 '
 '나라"에서 "유학하기 가장 좋은 나라"로 분류하는 데있어 최고의 자리를 보장하는 동시에 최고의 국가 일, 방문 및 생활로 자리 '
 '매김했습니다. 당신의 배낭에 단풍잎을 꿰매는 진정한 동기를 찾고 있습니까?')
('그러나 최근 몇 년 동안 점점 더 인기를 얻고있는 한 가지 선택이 있습니다. 캐나다 대학에서 교육을 추구하는 것입니다. 많은 유학생들이 '
 '캐나다를 유학 목적지로 선택하고 있으며 그 이유는 당연합니다. 캐나다는 우수한 교육 프로그램, 경이로운 자연, 다문화 사회 및 저렴한 '
 '생활비를 제공합니다.')


In [37]:
# RAPTOR RAG 답안 뽑기:"캐나다가 주목받는 국가인 이유에 대해 알려줘"
response =raptor_query_engine.query("캐나다가 주목받는 국가인 이유에 대해 알려줘")

In [38]:
pprint(response.response)

('캐나다는 3년 연속 세계에서 가장 주목할 만한 국가로 선정되었으며, "살기 좋은 나라"와 "유학하기 가장 좋은 나라"로도 높은 평가를 '
 '받고 있습니다. 이러한 평가는 캐나다의 교육 시스템이 세계적으로 인정받고 있고, 약 96개의 대학이 국제 교육을 고려하는 학생들에게 '
 '매력적인 선택지로 자리 잡고 있기 때문입니다. 또한, 캐나다는 다양한 국제 커뮤니티와의 연결이 중요하며, 공식 언어로 영어와 프랑스어를 '
 '사용하고 있습니다.')


In [39]:
# 참조 컨텍스트 확인
pprint(response.source_nodes[0].text)
pprint(response.source_nodes[1].text)

('캐나다는 3 년 연속 지구상에서 가장 주목할만한 국가로 선포되었습니다. 일반 타이틀을 집으로 가져 왔을뿐만 아니라 캐나다는 "살기 좋은 '
 '나라"에서 "유학하기 가장 좋은 나라"로 분류하는 데있어 최고의 자리를 보장하는 동시에 최고의 국가 일, 방문 및 생활로 자리 '
 '매김했습니다. 당신의 배낭에 단풍잎을 꿰매는 진정한 동기를 찾고 있습니까?')
('캐나다는 3년 연속 세계에서 가장 주목할 만한 국가로 선정되었으며, "살기 좋은 나라"와 "유학하기 가장 좋은 나라"로도 높은 평가를 '
 '받고 있습니다. 캐나다에는 약 96개의 대학이 있으며, 국제 교육을 고려하는 학생들에게 매력적인 선택지로 자리 잡고 있습니다. 캐나다의 '
 '교육 시스템은 세계적으로 인정받고 있으며, 2018년에는 세계에서 가장 국제적인 200개 대학 중 7개가 캐나다 대학으로 포함되었습니다. '
 '캐나다의 공식 언어는 영어와 프랑스어이며, 각각의 모국어 사용 비율은 56.9%와 21.3%입니다. 학생들은 두 언어를 모두 알 필요는 '
 '없지만, 다양한 국제 커뮤니티와의 연결이 중요할 수 있습니다.')


In [40]:
# Naive RAG 답안 뽑기:"한국과 일본의 유명한 해군 제독에 대해 알려줘"
naive_response =query_engine_200.query("한국과 일본의 유명한 해군 제독에 대해 알려줘")

In [41]:
pprint(naive_response.response)

('한국과 일본의 유명한 해군 제독에 대한 정보는 제공되지 않았습니다. 다만, 일본의 해군 제독 중 한 명인 마카로프 제독이 언급되었으며, '
 '그는 러시아 해군의 전략가로서 1904년 일러전쟁 중에 전사한 사례가 있습니다. 한국과 일본의 해군 제독에 대한 구체적인 정보는 포함되어 '
 '있지 않습니다.')


In [42]:
# 참조 컨텍스트 확인
pprint(naive_response.source_nodes[0].text)
pprint(naive_response.source_nodes[1].text)

('일본으로부터 막대한 영향을 받고 영향을 주기도 하며 서로가 공존하는 모습을 두고 한국은 선, 일본은 악이라는 이분법적 조센징의 머가리 '
 '구조가 우스꽝 스러워 보일뿐이다.')
('당시 러시아는 세계에서 영국에 이은 해군 강국으로 해군의 어뢰정 운용과 전술에 있어 최고로 인정 받고 있었으며, 해군 전술 전략에 있어 '
 '세계적으로 존경을 받고 있는 마카로프 제독과 같은 전략가가 있었다. 마카로프 제독은 1904년 일러전쟁 발발 직 후 여순항 해전 중 일본 '
 '연함 함대를 발견하고 쫓아가다 기뢰에 걸려 파편에 맞아 즉사했다.')


In [43]:
# RAPTOR RAG 답안 뽑기:"한국과 일본의 유명한 해군 제독에 대해 알려줘"
response =raptor_query_engine.query("한국과 일본의 유명한 해군 제독에 대해 알려줘")

In [44]:
pprint(response.response)

('한국과 일본의 유명한 해군 제독으로는 이순신과 도고 헤이하치로가 있습니다. 이순신은 임진왜란 동안 한국 해군을 이끌며 한산도 해전에서 '
 "일본에 결정적인 승리를 거두었습니다. 그의 독창적인 전투 형식인 '학익진'은 이 전투에서 큰 역할을 했습니다. 반면, 도고 헤이하치로는 "
 '러일 전쟁에서 일본 해군을 이끌며 발틱 함대를 크게 무찌른 전투로 유명합니다. 그는 이순신의 전술을 존경하며, 일본 해군의 현대화에 '
 '이순신의 전략을 연구했습니다. 두 제독 모두 각국의 해군 역사에서 중요한 인물로 평가받고 있습니다.')


In [45]:
# 참조 컨텍스트 확인
pprint(response.source_nodes[0].text)
pprint(response.source_nodes[1].text)

('The text covers several historical and contemporary aspects of Japan and '
 'South Korea, focusing on military history, economic comparisons, and '
 'cultural influences. \n'
 '\n'
 'The Tsushima Naval Battle during the Russo-Japanese War marked a significant '
 'victory for Japan, showcasing its military prowess by decimating the Russian '
 'Baltic Fleet. Japan suffered minimal casualties compared to Russia, which '
 "lost a substantial number of ships and personnel. Admiral Togo Heihachiro's "
 'innovative tactics, inspired by Admiral Yi Sun-sin of Korea, are noted as '
 'foundational in modern naval warfare.\n'
 '\n'
 'In contrast, during the Imjin War, Admiral Yi Sun-sin led the Korean navy to '
 'a decisive victory against Japan, employing the effective "crane wing '
 'formation." Despite his contributions, Yi\'s legacy was largely overlooked '
 'in Korea, while Japan revered him, studying his strategies to enhance their '
 'naval capabilities.\n'
 '\n'
 'The text also addresses